In [21]:
import numpy as np
import pandas as pd
from xml.etree import ElementTree as ET

### Warm-up exercise

In [22]:
membership = ET.parse('./data/data_wrangling_xml/data/membership.xml')

In [23]:
users = membership.find('users')

In [24]:
for user in membership.findall('users/user'):
    print(user.attrib['name'])

john
charles
peter


In [25]:
for group in membership.findall('groups/group'):
    print(group.attrib['name'])

users
administrators


In [27]:
for group in membership.findall('groups/group'):
    print('group:' , group.attrib['name'])
    print('users:')
    for node in group.getchildren():
        if node.tag == 'user':
            print('-', node.attrib['name'])

group: users
users:
- john
- charles
group: administrators
users:
- peter


In [29]:
#1. iterate nodes including the starting point
users = membership.find('users')
for node in users.getiterator():
    print(node.tag,node.attrib,node.text,node.tail)

users {} 
         

    
user {'name': 'john'} None 
        
user {'name': 'charles'} None 
        
user {'name': 'peter'} None 
    


In [31]:
#2. iterate only the children
users = membership.find('users')
for node in users.getchildren():
    print(node.tag,node.attrib,node.text,node.tail)

user {'name': 'john'} None 
        
user {'name': 'charles'} None 
        
user {'name': 'peter'} None 
    


# XML exercise
#### Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

## 1. 10 countries with the lowest infant mortality rates
## 2. 10 cities with the largest population
## 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
## 4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [122]:
mondial = ET.parse( './data/data_wrangling_xml/data/mondial_database.xml' )
mondial_less = ET.parse( './data/data_wrangling_xml/data/mondial_database_less.xml' )

In [79]:
type(mondial)

xml.etree.ElementTree.ElementTree

In [80]:
countries = mondial.find('country')

In [81]:
for c in countries:
    for node in c.getiterator():
        if node.tag == 'name':
            print('-',node.text)
##not the expected behavior....

- Albania
- Tirana
- Tirane
- Shkodër
- Durrës
- Vlorë
- Elbasan
- Korçë


In [96]:
df1 = pd.DataFrame(columns=('country','infant_mortality'))
d = {'country':'','infant_mortality':''}
cou = 0
for c in mondial.findall('country'):
    for n in c.getchildren():
        if n.tag == 'name':
            #print(cou, n.text)
            d['country'] = n.text
        if n.tag == 'infant_mortality':
            #print(n.text)
            d['infant_mortality'] = n.text
            #print(cou, d)
            df1.loc[cou] = d
            cou += 1

In [97]:
df1.head()

,country,infant_mortality
0,Albania,13.19
1,Greece,4.78
2,Macedonia,7.9
3,Serbia,6.16
4,Andorra,3.69


# 1. 10 countries with the lowest infant mortality rates

In [98]:
df1.sort_values('infant_mortality', ascending=True).head(10)

,country,infant_mortality
36,Monaco,1.81
28,Romania,10.16
142,Fiji,10.2
63,Brunei,10.48
124,Grenada,10.5
221,Mauritius,10.59
116,Panama,10.7
227,Seychelles,10.77
94,United Arab Emirates,10.92
105,Barbados,10.93


In [201]:
df2 = pd.DataFrame(columns=('country', 'province', 'city', 'population'))
d = {'country':'', 'province':'', 'city':'', 'population':''}
cou = 0
for c in mondial.findall('country'):
    for n in c.getchildren():
        if n.tag == 'name':
            #print(n.text)
            d['country'] = n.text
        if n.tag == 'city':
            for nd in n.getchildren():
                if nd.tag == 'name':
                    #print(cou,'-',nd.text)
                    d['city'] = nd.text
                if nd.tag == 'population':
                    if nd.attrib['year'] == '2011':
                        d['population'] = nd.text
                        d['province'] = 'none'
                df2.loc[cou] = d
            cou += 1
        if n.tag == 'province':
            d['province'] = n.getchildren()[0].text
            #print('province:', n.getchildren()[0].text)
            for c in n.findall('city'):
                for nd in c.getchildren():
                    if nd.tag == 'name':
                        #print(cou, '-',nd.text)
                        d['city'] = nd.text
                    if nd.tag == 'population':
                        if nd.attrib['year'] == '2011':
                            d['population'] = nd.text
                            #print('pop:',nd.attrib['year'], nd.text)
                    df2.loc[cou] = d
                cou += 1

In [202]:
df2.head(20)

,country,province,city,population
0,Albania,none,Tirane,418495
1,Albania,none,Shkodër,77075
2,Albania,none,Durrës,113249
3,Albania,none,Vlorë,79513
4,Albania,none,Elbasan,78703
5,Albania,none,Korçë,51152
6,Greece,Anatolikis Makedonias kai Thrakis,Komotini,51152
7,Greece,Anatolikis Makedonias kai Thrakis,Kavala,58790
8,Greece,Attikis,Athina,664046
9,Greece,Attikis,Piräus,163688


# 2. 10 cities with the largest population

In [204]:
df2.sort_values('population', ascending=True)

,country,province,city,population
316,Italy,Puglia,Andria,100052
96,Spain,País Vasco,Barakaldo,100064
3182,Botswana,none,Francistown,100079
987,United Kingdom,West Midlands,Worcester,100153
981,United Kingdom,East Midlands,Lincoln,100160
1554,India,Rajasthan,Ajmer,1001694
1555,India,Sikkim,Gangtok,1001694
1552,India,Rajasthan,Kota,1001694
1553,India,Rajasthan,Bikaner,1001694
293,Italy,Emilia-Romagna,Piacenza,100311
